Jupyter notebook för att göra kluster för Livsmedelsverkets databas

In [68]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [69]:
# Variable to toggle printouts
debug = True

Vi har konverterat en nedladdad Excel-fil till csv och sen till Sqlite 3. 

Den innehåller alla Livsmedelsverkets 59 näringsämnen. Här jobbar vi bara med kolhydrater, fett och protein.

Först läser vi in från databasen och lägger värdena i arrayen *result*

In [102]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
for row in curs.execute('select "Kolhydrater_g", "Fett_g", "Protein_g", "Vatten_g" from livs limit 10'):
        result.append(row)
conn.close()

För klustringsalgoritmerna behöver vi använda numpy och göra en np-array

För fem livsmedel, kolumner för kolhydrater, fett och protein

<pre>[[   0.    71.     7. ]
 [   0.    85.     2.8]
 [   0.   100.     0. ]
 [   0.   100.     0. ]
 [   0.5   80.     0.5]]</pre>



In [103]:
dataSet = np.array(result)
if debug:
    print dataSet

[[   0.     71.      7.     21.7 ]
 [   0.     85.      2.8    11.1 ]
 [   0.    100.      0.      0.  ]
 [   0.    100.      0.      0.  ]
 [   0.5    80.      0.5    17.47]
 [   0.5    60.      0.4    38.1 ]
 [   0.3    70.      0.2    28.3 ]
 [   0.3    60.      0.2    38.3 ]
 [   0.     60.      0.     39.1 ]
 [   0.     82.      0.     16.7 ]]


Funktion för att göra själva klustringen enligt k-means.

K-means förklaras relativt enkelt här: http://bigdata-madesimple.com/possibly-the-simplest-way-to-explain-k-means-algorithm/

In [104]:
def cluster(dataSet):

    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    assignedCluster = kmeans.labels_

    if debug:
        print (centroids)
        print
        print (assignedCluster)

    return assignedCluster

När man kör klustringsalgoritmen returneras en array av vilket kluster varje livsmedel tillhör. 

Man kan även få ut mittpunkterna i varje kluster genom variabeln *centroids*

Ett exempel: 

<pre>
centroids:
[[   0.    100.      0.  ]
 [   0.25   82.5     1.65]
 [   0.     71.      7.  ]]

assignedCluster:
[2 1 0 0 1]</pre>

In [105]:
numClusters = 5
assignedCluster = cluster(dataSet)

[[   0.26666667   60.            0.2          38.5       ]
 [   0.16666667   82.33333333    1.1          15.09      ]
 [   0.          100.            0.            0.        ]
 [   0.3          70.            0.2          28.3       ]
 [   0.           71.            7.           21.7       ]]

[4 1 2 2 1 0 3 0 0 1]


I princip kan det här göras med många fler näringsvärden där varje näringsvärde blir en "dimension". 

Eftersom vi vill visa klustringen i ett 3D-diagram har vi begränsat oss till tre dimensioner i exemplet.

## Visualisering med Plotly

I Plotly svarar varje näringsvärde mot en axel.

För att Plotly ska kunna plotta varje axel för sig transponerar vi data så vi får en rad för x-axeln, en för y-axeln och en för z.

Ett exempel:

<pre>
[[   0.     0.     0.     0.     0.5]
 [  71.    85.   100.   100.    80. ]
 [   7.     2.8    0.     0.     0.5]]
</pre>

In [106]:
dataToChart = dataSet.transpose()

In [107]:
if debug:
    print dataToChart

[[   0.      0.      0.      0.      0.5     0.5     0.3     0.3     0.
     0.  ]
 [  71.     85.    100.    100.     80.     60.     70.     60.     60.
    82.  ]
 [   7.      2.8     0.      0.      0.5     0.4     0.2     0.2     0.
     0.  ]
 [  21.7    11.1     0.      0.     17.47   38.1    28.3    38.3    39.1
    16.7 ]]


För att visualisera vilka kluster som ett livsmedel ligger så ger vi varje kluster olika färger. I Plotly svarar detta mot att varje kluster hamnar i ett separat "trace"

Därför behöver vi gå igenom vårt *dataToChart* och separera ut vilka livsmedel som hör till vilket kluster.

Då behöver vi en tvådimensionell array där storleken på första dimensionen är antal kluster och storleken på andra dimensionen är tre (eftersom vi visualiserar tre dimensioner)

In [124]:
# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(4)] for _ in range(numClusters)]


Det är lite krångligt att visualisera den här strukturen. Så här blir den med fem kluster (numClusters) och tre faktorer för klustringen (fett, protein, kolhydrater) 
<pre>
[
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []]
]
</pre>

(Om man väljer att klustra med fler än tre faktorer är det svårt att visualisera alla, utan man måste välja bort någon faktor)

In [125]:
jobbigArray = np.array(segmentedData)
print (jobbigArray)

[]


In [133]:
for num, cluster in enumerate(assignedCluster):
        if debug:
            print (str(num) + ' ' + str(cluster))
        segmentedData[cluster][0].append(dataToChart[0][num])
        segmentedData[cluster][1].append(dataToChart[1][num])
        segmentedData[cluster][2].append(dataToChart[2][num])
        segmentedData[cluster][3].append(dataToChart[3][num])
        
jobbigArray = np.array(segmentedData)
print (jobbigArray.ndim)

0 4
1 1
2 2
3 2
4 1
5 0
6 3
7 0
8 0
9 1
2


In [134]:
if debug:
    print(segmentedData)

[[[0.5, 0.29999999999999999, 0.0, 0.5, 0.29999999999999999, 0.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0], [0.40000000000000002, 0.20000000000000001, 0.0, 0.40000000000000002, 0.20000000000000001, 0.0], [38.100000000000001, 38.299999999999997, 39.100000000000001]], [[0.0, 0.5, 0.0, 0.0, 0.5, 0.0], [85.0, 80.0, 82.0, 85.0, 80.0, 82.0], [2.7999999999999998, 0.5, 0.0, 2.7999999999999998, 0.5, 0.0], [11.1, 17.469999999999999, 16.699999999999999]], [[0.0, 0.0, 0.0, 0.0], [100.0, 100.0, 100.0, 100.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0]], [[0.29999999999999999, 0.29999999999999999], [70.0, 70.0], [0.20000000000000001, 0.20000000000000001], [28.300000000000001]], [[0.0, 0.0], [71.0, 71.0], [7.0, 7.0], [21.699999999999999]]]


In [135]:
# create traces for plotly
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][3],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1


In [136]:
layout = go.Layout(
        scene=go.Scene(
            xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
            yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
            zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
        ),
        height = 800,
        width = 800,
    )

In [137]:
fig = go.Figure(data=traces, layout=layout)    

In [138]:
py.iplot(fig, filename='table1')